![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_CamemBertForSequenceClassification.ipynb)

## Import ONNX CamemBertForSequenceClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `CamemBertForSequenceClassification` is only available since in `Spark NLP 5.2.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import CamemBERT models trained/fine-tuned for sequence classification via `CamembertForSequenceClassification` or `TFCamembertForSequenceClassification`. These models are usually under `Sequence Classification` category and have `camembert` in their labels
- Reference: [TFCamembertForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/camembert#transformers.TFCamembertForSequenceClassification)
- Some [example models](https://huggingface.co/models?other=camembert&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.4`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- CamembertTokenizer requires the `SentencePiece` library, so we install that as well

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.52.4 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [tblard/tf-allocine](https://huggingface.co/tblard/tf-allocine)  model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.
- In addition to the CamemBERT model, we also need to save the `CamembertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import CamembertTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

MODEL_NAME = "astrosbd/french_emotion_camembert"
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL)

tokenizer = CamembertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(ONNX_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/374 [00:00<?, ?B/s]

('onnx_models/astrosbd/french_emotion_camembert/tokenizer_config.json',
 'onnx_models/astrosbd/french_emotion_camembert/special_tokens_map.json',
 'onnx_models/astrosbd/french_emotion_camembert/sentencepiece.bpe.model',
 'onnx_models/astrosbd/french_emotion_camembert/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {ONNX_MODEL}

total 433260
-rw-r--r-- 1 root root        22 Jun 11 21:26 added_tokens.json
-rw-r--r-- 1 root root       971 Jun 11 21:26 config.json
-rw-r--r-- 1 root root 442825702 Jun 11 21:26 model.onnx
-rw-r--r-- 1 root root    810912 Jun 11 21:26 sentencepiece.bpe.model
-rw-r--r-- 1 root root      1058 Jun 11 21:26 special_tokens_map.json
-rw-r--r-- 1 root root      1813 Jun 11 21:26 tokenizer_config.json


- We need to move `sentencepiece.bpe.model` from the tokenizer to assets folder which Spark NLP will look for
- In addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [4]:
!mkdir -p {ONNX_MODEL}/assets && mv {ONNX_MODEL}/sentencepiece.bpe.model {ONNX_MODEL}/assets/

with open(f"{ONNX_MODEL}/assets/labels.txt", "w") as f:
    f.write("\n".join([v for _, v in sorted(ort_model.config.id2label.items())]))

In [5]:
!ls -lR {ONNX_MODEL}

onnx_models/astrosbd/french_emotion_camembert:
total 432472
-rw-r--r-- 1 root root        22 Jun 11 21:26 added_tokens.json
drwxr-xr-x 2 root root      4096 Jun 11 21:26 assets
-rw-r--r-- 1 root root       971 Jun 11 21:26 config.json
-rw-r--r-- 1 root root 442825702 Jun 11 21:26 model.onnx
-rw-r--r-- 1 root root      1058 Jun 11 21:26 special_tokens_map.json
-rw-r--r-- 1 root root      1813 Jun 11 21:26 tokenizer_config.json

onnx_models/astrosbd/french_emotion_camembert/assets:
total 796
-rw-r--r-- 1 root root     35 Jun 11 21:26 labels.txt
-rw-r--r-- 1 root root 810912 Jun 11 21:26 sentencepiece.bpe.model


In [6]:
!cat {ONNX_MODEL}/assets/labels.txt

sad
fear
anger
neutral
surprise
joy

Voila! We have our `sentencepiece.bpe.model` and `labels.txt` inside assets directory

## Import and Save CamemBertForSequenceClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab.
- For this example we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly.

In [7]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 41.0 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `CamemBertForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `CamemBertForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [9]:
from sparknlp.annotator import CamemBertForSequenceClassification

sequenceClassifier = CamemBertForSequenceClassification.loadSavedModel(
     f"{ONNX_MODEL}",
     spark
 )\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your CamemBertForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [12]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 433320
-rw-r--r-- 1 root root 442893406 Jun 11 21:28 camembert_classification_onnx
-rw-r--r-- 1 root root    810912 Jun 11 21:28 camembert_spp
drwxr-xr-x 3 root root      4096 Jun 11 21:28 fields
drwxr-xr-x 2 root root      4096 Jun 11 21:28 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBertForSequenceClassification model 😊

In [13]:
sequenceClassifier_loaded = CamemBertForSequenceClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [14]:
sequenceClassifier_loaded.getClasses()

['surprise', 'joy', 'anger', 'neutral', 'sad', 'fear']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [15]:
example = spark.createDataFrame([
    ["Je me sens seul et personne ne me comprend.", "sad"],
    ["Je ne sais pas ce qui va se passer, ça m'inquiète.", "fear"],
    ["Tu m'as vraiment déçu, je suis furieux.", "anger"],
    ["Je n'ai pas d'opinion particulière sur ce sujet.", "neutral"],
    ["Quoi ? Tu as gagné à la loterie ?", "surprise"],
    ["Je suis tellement content de te revoir !", "joy"]
], ["text", "expected_emotion"])

example.show(truncate=False)

+--------------------------------------------------+----------------+
|text                                              |expected_emotion|
+--------------------------------------------------+----------------+
|Je me sens seul et personne ne me comprend.       |sad             |
|Je ne sais pas ce qui va se passer, ça m'inquiète.|fear            |
|Tu m'as vraiment déçu, je suis furieux.           |anger           |
|Je n'ai pas d'opinion particulière sur ce sujet.  |neutral         |
|Quoi ? Tu as gagné à la loterie ?                 |surprise        |
|Je suis tellement content de te revoir !          |joy             |
+--------------------------------------------------+----------------+



In [16]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.sql.functions import col

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

result = pipeline.fit(example).transform(example)
result.withColumn("predicted_emotion", col("class.result")[0]) \
    .select("text", "expected_emotion", "predicted_emotion") \
    .show(truncate=False)

+--------------------------------------------------+----------------+-----------------+
|text                                              |expected_emotion|predicted_emotion|
+--------------------------------------------------+----------------+-----------------+
|Je me sens seul et personne ne me comprend.       |sad             |sad              |
|Je ne sais pas ce qui va se passer, ça m'inquiète.|fear            |fear             |
|Tu m'as vraiment déçu, je suis furieux.           |anger           |anger            |
|Je n'ai pas d'opinion particulière sur ce sujet.  |neutral         |neutral          |
|Quoi ? Tu as gagné à la loterie ?                 |surprise        |sad              |
|Je suis tellement content de te revoir !          |joy             |joy              |
+--------------------------------------------------+----------------+-----------------+



That's it! You can now go wild and use hundreds of `CamemBertForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
